In [1]:
import pymysql.cursors
from datetime import datetime
import matplotlib.pyplot as plt 
import pandas as pd
import math
import numpy as np
from scipy import spatial

In [2]:
def to_dict(keys, values):
    return dict(zip(keys, values))

In [66]:
class Graph:
    '''
    +------------------+
    | edge_type        |
    +----+-------------+
    | id | description |
    +----+-------------+
    |  0 | Email       |
    |  1 | Phone       |
    |  2 | Sell        |
    |  3 | Buy         |
    |  4 | Authro-of   |
    |  5 | Financial   |
    |  6 | Travels-to  |
    +----+-------------+
    
    +------------------------------------------------------------------+
    | node_type                                                        |
    +----+--------------------+----------------------------------------+
    | id | description        | used_in                                |
    +----+--------------------+----------------------------------------+
    |  1 | Person             | Used in all channels                   |
    |  2 | Product category   | For procurements, etype = 3            |
    |  3 | Document           | For the co-authorship graph, etype = 4 |
    |  4 | Financial category | For financial demographics, etype = 5  |
    |  5 | Country            | For travel, etype = 6                  |
    +----+--------------------+----------------------------------------+
    
    +-----------------------------------------------|
    | demographic_categories                        |
    +---------+-------------------------------------+
    | node_id | category                            |
    +---------+-------------------------------------+
    |  459381 | Water and other public services     |
    |  466907 | Electricity                         |
    |  473173 | Household furnishings               |
    |  503218 | Natural gas                         |
    |  503701 | Miscellaneous                       |
    |  510031 | Gifts                               |
    |  520660 | Healthcare                          |
    |  523927 | Restaurants                         |
    |  527449 | Alcohol                             |
    |  536346 | Home maintenance                    |
    |  537281 | Housekeeping supplies               |
    |  552988 | Money income before taxes           |
    |  567195 | Personal insurance and pensions     |
    |  571970 | Reading                             |
    |  575030 | Transportation                      |
    |  577992 | Education                           |
    |  580426 | Telephone services                  |
    |  589943 | Lodging away from home              |
    |  595298 | Groceries                           |
    |  595581 | Donations                           |
    |  606730 | Entertainment                       |
    |  616315 | Apparel and services                |
    |  620120 | Personal taxes                      |
    |  621924 | Mortgage payments                   |
    |  630626 | Rented dwellings                    |
    |  632961 | Personal care products and services |
    |  640784 | Tobacco                             |
    |  642329 | Household operations                |
    |  644226 | Property taxes                      |
    +---------+-------------------------------------+
    '''
    def __init__(self):
        self.financial_type = {
             459381: 'Water and other public services',
             466907: 'Electricity',
             473173: 'Household furnishings',
             503218: 'Natural gas',
             503701: 'Miscellaneous',
             510031: 'Gifts',
             520660: 'Healthcare',
             523927: 'Restaurants',
             527449: 'Alcohol',
             536346: 'Home maintenance',
             537281: 'Housekeeping supplies',
             552988: 'Money income before taxes',
             567195: 'Personal insurance and pensions',
             571970: 'Reading',
             575030: 'Transportation',
             577992: 'Education',
             580426: 'Telephone services',
             589943: 'Lodging away from home',
             595298: 'Groceries',
             595581: 'Donations',
             606730: 'Entertainment',
             616315: 'Apparel and services',
             620120: 'Personal taxes',
             621924: 'Mortgage payments',
             630626: 'Rented dwellings',
             632961: 'Personal care products and services',
             640784: 'Tobacco',
             642329: 'Household operations',
             644226: 'Property taxes'
        }
        self.edge_type = { 0: 'Email', 1: 'Phone', 2: 'Sell', 3: 'Buy', 4: 'Authro-of', 5: 'Financial', 6: 'Travels-to' }
        self.node_type = { 1: 'Person', 2: 'Product category', 3: 'Document', 4: 'Financial category', 5: 'Country' }
    
    
    def find_nodes_from_source_node(self, node_id):
        return self.get_by_params(source=node_id)


    def find_nodes_from_target_node(self, node_id):
        return self.get_by_params(target=node_id)
    
    
    def find_nodes_from_source_node_and_edge_type(self, node_id, edge_type):
        return self.get_by_params(source=node_id, e_type=edge_type)
    
    
    def find_nodes_from_target_node_and_edge_type(self, node_id, edge_type):
        return self.get_by_params(target=node_id, e_type=edge_type)
    
    
    def find_node_type(self, node_id):
        pass
    
    
    def find_by_params(self,
                      source=None,
                      e_type=None,
                      target=None,
                      time=None,
                      weight=None,
                      source_location=None,
                      target_location=None,
                      source_latitude=None,
                      source_longitude=None,
                      target_latitude=None,
                      target_longitude=None):
        pass
    
    
    def node_type_description(self, type_id):
        return self.node_type[type_id]
    
    
    def edge_type_description(self, type_id):
        return self.edge_type[type_id]
    
    
    def financial_description(self, type_id):
        return self.financial_type[type_id]
    
    
    def all_financial_types(self):
        return self.financial_type
    
    
    def build_financial_dict(self, node_id):
        '''
        Take the relations between a node and their financial links, the noda could be source or target.
        Translate the union in a dictionaty with the id of the financial category like key and the weight as value.
        '''
        filter_data_source = self.find_by_params(source=node_id, e_type=5)[['target', 'weight']].rename(columns={"target": "financial", "weight": "value"})
        filter_data_target = self.find_by_params(target=node_id, e_type=5)[['source', 'weight']].rename(columns={"source": "financial", "weight": "value"})
        filter_data = pd.concat([filter_data_source, filter_data_target], axis=0)
        
        return dict(zip(filter_data.financial, filter_data.value))

In [4]:
class GraphMysql(Graph):
    '''
    +------------------------+
    | Tables_in_vast         |
    +------------------------+
    | demographic_categories |
    | edge_type              |
    | links                  |
    | node_type              |
    | nodes                  |
    +------------------------+
    '''
    def __init__(self, connection_config):
        self.hostname = connection_config['hostname']
        self.db_user = connection_config['user']
        self.db_password = connection_config['password']
        self.db_schema = connection_config['schema']
        self.db_port = connection_config['port']
        self.connection = pymysql.connect(host=self.hostname,
                             user=self.db_user,
                             password=self.db_password,
                             db=self.db_schema,
                             charset='utf8mb4',
                             port=self.db_port,
                             cursorclass=pymysql.cursors.DictCursor)
        self.links = 'links'
        self.nodes = 'nodes'
        super().__init__()
        
        
    def execute_sql_query(self, sql):
        self.connection.ping(reconnect=True)
        try:
            with self.connection.cursor() as cursor:
                cursor.execute(sql)
                result = cursor.fetchall()
        finally:
            self.connection.close()
        return result
    
    
    def execute_sql_query_unique(self, sql):
        self.connection.ping(reconnect=True)
        try:
            with self.connection.cursor() as cursor:
                cursor.execute(sql)
                result = cursor.fetchone()
                if cursor.fetchone() != None:
                    raise Exception('Error, must be a unique result')
                
        finally:
            self.connection.close()
        return result
    
    
    def find_node_type(self, node_id):
        sql = f'SELECT * FROM {self.nodes} where id = {node_id}'
        return self.execute_sql_query_unique(sql)['node_type']
    
    
    def find_by_params(self,
                      source=None,
                      e_type=None,
                      target=None,
                      time=None,
                      weight=None,
                      source_location=None,
                      target_location=None,
                      source_latitude=None,
                      source_longitude=None,
                      target_latitude=None,
                      target_longitude=None):
        sql = f'SELECT * FROM {self.links}'
        conditions = []
        
        if source != None:
            conditions.append(f'source = {source}')
        if e_type != None:
            conditions.append(f'e_type = {e_type}')
        if target != None:
            conditions.append(f'target = {target}')
        if time != None:
            conditions.append(f'time = {time}')
        if weight != None:
            conditions.append(f'weight = {weight}')
        if source_location != None:
            conditions.append(f'source_location = {source_location}')
        if target_location != None:
            conditions.append(f'target_location = {target_location}')
        if source_latitude != None:
            conditions.append(f'source_latitude = {source_latitude}')
        if source_longitude != None:
            conditions.append(f'source_longitude = {source_longitude}')
        if target_latitude != None:
            conditions.append(f'target_latitude = {target_latitude}')
        if target_longitude != None:
            conditions.append(f'target = {target_longitude}')
        
        if len(conditions) > 0:
            sql += f' WHERE {" AND ".join(conditions)}'

        return pd.DataFrame(self.execute_sql_query(sql))

In [5]:
class GraphCsv(Graph):
    def __init__(self, nodes_file_name, links_file_name):
        self.nodes = pd.read_csv(nodes_file_name ,sep=',')
        self.links = pd.read_csv(links_file_name, sep=',')
        super().__init__()
    
    
    def find_node_type(self, node_id):
        return self.nodes[(self.nodes.id == node_id)].iloc[0]['node_type']
    
    
    def find_by_params(self,
                      source=None,
                      e_type=None,
                      target=None,
                      time=None,
                      weight=None,
                      source_location=None,
                      target_location=None,
                      source_latitude=None,
                      source_longitude=None,
                      target_latitude=None,
                      target_longitude=None):
        partial_filter = pd.DataFrame(self.links)
        if source != None:
            partial_filter = partial_filter[(partial_filter.source == source)]
        if e_type != None:
            partial_filter = partial_filter[(partial_filter.e_type == e_type)]
        if target != None:
            partial_filter = partial_filter[(partial_filter.target == target)]
        if time != None:
            partial_filter = partial_filter[(partial_filter.time == time)]
        if weight != None:
            partial_filter = partial_filter[(partial_filter.weight == weight)]
        if source_location != None:
            partial_filter = partial_filter[(partial_filter.source_location == source_location)]
        if target_location != None:
            partial_filter = partial_filter[(partial_filter.target_location == target_location)]
        if source_latitude != None:
            partial_filter = partial_filter[(partial_filter.source_latitude == source_latitude)]
        if source_longitude != None:
            partial_filter = partial_filter[(partial_filter.source_longitude == source_longitude)]
        if target_latitude != None:
            partial_filter = partial_filter[(partial_filter.target_latitude == target_latitude)]
        if target_longitude != None:
            partial_filter = partial_filter[(partial_filter.target_longitude == target_longitude)]
            
        return partial_filter

In [6]:
class FinancialProfile:
    def __init__(self, financial_categories, financial_values, node_id):
        self.node_id = node_id
        self.financial_vector = []
        
        # Each position in the vector represents a financial category, if the value is not present then fill with 0.
        # The values in every position represents the same financial attribute for every FinancialProfile. 
        for key in sorted(financial_categories.keys()):
            if key in financial_values.keys():
                self.financial_vector.append(float(financial_values[key]))
            else:
                self.financial_vector.append(0.0)
    
    
    def cosine_simalarity(self, another_profile):
        return (1 - spatial.distance.cosine(self.financial_vector, another_profile.financial_vector))

In [7]:
template_graph = GraphCsv('./template_nodes.csv', './template_links.csv')
seed_1 = {'source': 600971, 'e_type': 4, 'target': 579269, 'time': -685755382, 'weight': 0.166667, 'source_location': 0, 'target_location': 0, 'source_latitude': 0, 'source_longitude': 0, 'target_latitude': 0, 'target_longitude': 0}
seed_2 = {'source': 538771, 'e_type': 4, 'target': 473043, 'time': -623491200, 'weight': 0.0909091, 'source_location': 0, 'target_location': 0, 'source_latitude': 0, 'source_longitude': 0, 'target_latitude': 0, 'target_longitude': 0}
seed_3 = {'source': 574136, 'e_type': 2, 'target': 657187, 'time': 1991785, 'weight': 633, 'source_location': 0, 'target_location': 0, 'source_latitude': 0, 'source_longitude': 0, 'target_latitude': 0, 'target_longitude': 0}

In [8]:
mysql_config = {'hostname': 'localhost', 'user': 'root', 'password': 'root', 'schema': 'vast', 'port': 52000}
large_graph = GraphMysql(mysql_config)

In [9]:
seed_1

{'source': 600971,
 'e_type': 4,
 'target': 579269,
 'time': -685755382,
 'weight': 0.166667,
 'source_location': 0,
 'target_location': 0,
 'source_latitude': 0,
 'source_longitude': 0,
 'target_latitude': 0,
 'target_longitude': 0}

In [33]:
template_graph.node_type_description(template_graph.find_node_type(0))

'Person'

In [34]:
large_graph.node_type_description(large_graph.find_node_type(600971))

'Person'

In [13]:
dict_financial = large_graph.build_financial_dict(600971)

In [14]:
profile = FinancialProfile(large_graph.all_financial_types(), dict_financial, 600971)

In [15]:
template_profile = FinancialProfile(template_graph.all_financial_types(), template_graph.build_financial_dict(0), 0)

In [16]:
profile.cosine_simalarity(template_profile)

0.9521792416337577

In [60]:
def most_similar(graph_template, graph, template_node_id, seed_node_id, e_type, side='source'):
    if 'source' == side:
        neighbors = graph.find_by_params(source=seed_node_id, e_type=e_type)['target'].unique()
        neighbors_template = graph_template.find_by_params(source=template_node_id, e_type=e_type)['target'].unique()
    elif 'target' == side:
        neighbors = graph.find_by_params(target=seed_node_id, e_type=e_type)['source'].unique()
        neighbors_template = graph_template.find_by_params(target=template_node_id, e_type=e_type)['source'].unique()
    
    if len(neighbors_template) == 0:
        return None
    
    # This map has all the financial profiles of the large graph
    profiles_by_node = {}
    for a_node in neighbors:
        profiles_by_node[a_node] = FinancialProfile(graph.all_financial_types(), graph.build_financial_dict(a_node), a_node)
    
    # This map has the best matching node for each node in the template
    matching_nodes = {}
    for a_node in neighbors_template:
        template_profile = FinancialProfile(graph_template.all_financial_types(), graph_template.build_financial_dict(a_node), a_node)
        sim = -1
        most_like = -1
        for k, v in profiles_by_node.items():
            # if the node k is in the matching_nodes skip it.
            if k in matching_nodes.values():
                continue
            new_sim = template_profile.cosine_simalarity(v)
            if new_sim > sim:
                sim = new_sim
                most_like = k
        matching_nodes[a_node] = most_like
        
    return matching_nodes
    

In [21]:
large_node_id = 600971
template_node_id = 0

In [22]:
mirror_nodes = {}

In [43]:
mirror_nodes[large_node_id] = template_node_id

In [44]:
e_type = 0

In [65]:
most_similar(template_graph, large_graph, mirror_nodes[large_node_id], large_node_id, 0, side='target')

{40: 595737, 41: 644620, 65: 621178}

In [67]:
most_similar(template_graph, large_graph, mirror_nodes[large_node_id], large_node_id, 1, side='target')

{41: 473963, 65: 488050, 40: 511468}